<a href="https://colab.research.google.com/github/JosepCristobal/NLPSearchPT/blob/master/NLPSearchPTv03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo.

Procedemos a los imports y las instalaciones necesarias

In [1]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=f48ea895e02274aed5148cff5f4e6da4262afdb316fd09fcb7f5667e39cdcc88
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [0]:
import spacy
import numpy as np

import pickle
import json
import os
import csv
import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import shuffle, choice, sample

from sklearn.model_selection import StratifiedShuffleSplit

from copy import copy

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load('en_core_web_sm')

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.


In [0]:
#realDonaldTrump
from google.colab import files
import io
import spacy
from stop_words import get_stop_words
from string import punctuation

In [0]:
stop_words = get_stop_words('en') + list(punctuation) + [' ']
nlp = spacy.load('en_core_web_sm')

In [0]:
import json

In [7]:
#Cargamos nuestro dataset desde nuestro diretorio local a través de la selección de fichero en cuadro de dialogo.
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving realDonaldTrump.json to realDonaldTrump.json
User uploaded file "realDonaldTrump.json" with length 524785 bytes


In [8]:
#En este caso, cargaremos un fichero en formato json
dt_names = uploaded['realDonaldTrump.json']
dt_names = json.loads(dt_names)
dt_names['10']


'Great day of meetings with Prime Minister @AbeShinzo of Japan! https://t.co/uKcdA8RxoN'

In [9]:
#Cargamos a nuestro dataframe df la información del json descargado
df=pd.DataFrame(dt_names.items())
df.head(5)

,0,1
0,0,My thoughts and prayers are with the families ...
1,1,My thoughts and prayers are with the families ...
2,2,I am heading for Canada and the G-7 for talks ...
3,3,Congratulations to the Washington Capitals on ...
4,4,Looking forward to straightening out unfair Tr...


In [0]:
#MAS LIMPIEZA
df[1] = df[1].str.strip()
#df.head(3)

In [0]:
df.dropna(subset=[1],inplace=True)
#df.head(3)

In [12]:
import re
import string
#Utilizamos regex para limpiar nuestro dataset de carácteres especiales, signos de puntuación , etc. 
def clean_text_round1(text):
    #Transformamos las palabras en minusculas, eliminamos los corchete, eliminamos signos de puntuación y palabras con números.
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text
 
round1 = lambda x: clean_text_round1(x)
 
data_clean = pd.DataFrame(df[1].apply(round1))
 
# Aplicamos una segunda pasada de limpieza
def clean_text_round2(text):
    #Eliminamos signos de puntuación adicionales que en la primera vuelta no fueron tratados
    text = re.sub('[‘’“”…«»]', '', text)
    #text = re.sub('\n', ' ', text)
    return text
 
round2 = lambda x: clean_text_round2(x)
 
data_clean = pd.DataFrame(data_clean[1].apply(round2))
data_clean.head(5)
 

,1
0,my thoughts and prayers are with the families ...
1,my thoughts and prayers are with the families ...
2,i am heading for canada and the g-7 for talks ...
3,congratulations to the washington capitals on ...
4,looking forward to straightening out unfair tr...


In [0]:
#Mas pruebas de Stop words
#print(pkg_stop_words)
#print(data_clean.head(10))

#from string import punctuation
pkg_stop_words = get_stop_words('en')

#data_clean[1].apply(lambda x: [item for item in x if item not in pkg_stop_words])
# No me funciona, me trata carácter a carácter y no consigo que lo haga por palabras.
#Estoy retrasando la práctica y lo dejo para el final, ya llevo demasiadas horas con ello. 
#print(data_clean.head(10))

#Lo he podido solucionar,
# aplicando el filtro de stop words en un entreno de 150 epochs hemos podido mejorar un poco, acc: 0.7423, 3 decimas repecto al anterior entreno

dt_dataset = list()
for index, row in data_clean.iterrows():
    if index > 0:
        sentence = row[1]
        #sentence_result=' '.join([word for word in sentence.split() if word not in (stopwords.words('english'))])
        sentence_result=' '.join([word for word in sentence.split() if word not in pkg_stop_words])
        dt_dataset.append(sentence_result)

#dt_dataset

In [17]:
from string import punctuation
dt_dataset = list()
for index, row in data_clean.iterrows():
    if index > 0:
        sentence = row[1]
        dt_dataset.append(sentence)

len(dt_dataset), dt_dataset[0]

(2994,
 'my thoughts and prayers are with the families of our serviceman who was killed and his fellow servicemen who were wounded in somalia. they are truly all heroes.')

In [0]:
# tokenizamos por caracteres
tokenized = [list(x) for x in dt_dataset] 

#Generamos embeddigs con carácteres igual que en el ejemplo de clase

In [20]:
# Nos quedamos con los 5 caracteres iniciales
init_chars = [x[:5] for x in tokenized]
print(init_chars[:3]) #Mostramos los 3 primeros 

[['t', 'h', 'o', 'u', 'g'], ['h', 'e', 'a', 'd', 'i'], ['c', 'o', 'n', 'g', 'r']]


In [0]:
for i in range(len(init_chars)):
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>')
    init_chars[i] = tmp[:5]

In [22]:
#Miramos la longitud de las frases, calculamos máximo y media
maxlen = max([len(x) for x in tokenized])
avglen = sum([len(x) for x in tokenized]) / len(tokenized)
maxlen, avglen

(267, 119.69539078156312)

In [23]:
#Queremos saber cuantos carácteres tenemos en nuestro vocabulario
total_tokens = [t for s in dt_dataset for t in s]
len(total_tokens)

358368

In [24]:
#Generamos el vocabulario
from collections import Counter
vocab_counter = Counter(total_tokens)
#filtramos los carácteres que mínimo hayan aparecido 2 veces
vocab = [w for w, v in vocab_counter.items() if v>2] 
#Añadimos cuatro tokens especiales a nuestro vocabulario
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab

nb_vocab = len(vocab)
nb_vocab

79

In [0]:
#Ahora generamos nuestros índices para los carácteres
c2id = {k: i for i, k in enumerate(vocab)}
id2c = {i: k for k, i in c2id.items()}

In [26]:
maxlen = 5
step = 1
data_train = []
# x es una lista con la frase "tokenizada"
for x in tokenized: 
    x.insert(0, '<SOS>')
    x.append('<EOS>')
    for i in range(0, len(x)-maxlen, step):
        data_train.append((x[i:i+maxlen], x[i+maxlen]))

len(data_train)

349386

In [27]:
data_train[:5]

[(['<SOS>', 't', 'h', 'o', 'u'], 'g'),
 (['t', 'h', 'o', 'u', 'g'], 'h'),
 (['h', 'o', 'u', 'g', 'h'], 't'),
 (['o', 'u', 'g', 'h', 't'], 's'),
 (['u', 'g', 'h', 't', 's'], ' ')]

Funciones de soporte. Utilizaremos los callbacks de Keras tal y como se ha explicado en clase

In [0]:
#Ayuda a que las predicciones de los carácteres ganen diversidad
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
#Cuando termina la epoch de entreno nos dará una muestra de lo que está aprendiendo.
#Vemos el progreso a nievel de frases generadas
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': len(vocab),
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2c[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))


In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

Montamos nuestra arquitectura y preparamos el Train y Predict

In [0]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
        
    def compile_model(self, params={}):
        #Primero, nuestra capa de Input y a continuación la de embedding
        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        embedding = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        
        #la CuDNNLSTM sólo se ejecuta en la GPU por lo tendremos que activarla en nuestro proyecto
        #return_sequences= true => one to many
        lstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')
        lmlstm = Bidirectional(lstm)(embedding)
        
        stacklstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stack')
        stackedlstm = stacklstm(lmlstm)

        out = Dense(params['vocab'], activation='softmax')(stackedlstm)

        model = Model(lm_inputs, out)
        model.compile(
            optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        #Displayamos el summary para hacer un análisis
        model.summary()
        return model
        
     #Implementamos el train y el predict   
    def train(self, model, data, params={}):
        callbacks = self._get_callbacks()

        if 'shuffle' in params and params['shuffle']:
            shuffle(data)

        sentences, next_chars = zip(*data)
        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = c2id[char] if char in c2id else c2id['<UNK>']
            y[i, c2id[next_chars[i]] if next_chars[i] in c2id else c2id['<UNK>']] = 1

        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)
        
    def predict(self, model, data, params={}):
        for diversity in [0.2, 0.6, 1.2]:
            print('------ diversity: ', diversity)
            sentence = copy(data)
            generated = copy(data)
            for i in range(len(data), 240): # si fuese un tweet
                x_pred = np.zeros((1, params['maxlen']))
                for t, char in enumerate(sentence):
                    x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                
                preds = self.model.predict(x_pred, verbose=0)[0]
                next_index = sample_pred(preds, diversity)
                next_char = id2c[next_index]

                if next_char == '<EOS>':
                    break
                
                generated += [next_char]
                sentence = sentence[1:]
                sentence+= [next_char]
            
            print(''.join(generated))
        
    def _get_callbacks(self, model_path='model_lm.h5'):
        #EarlyStopping detiene el entreno cuando la validation loss ya no baja
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)
        #ModelCheckpoint guarda el mejor modelo que va consiguiendo
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, period=2)
        #ReduceLROnPlateau ajusta la learning rate para conseguir mejores resultados.
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        st = Sampletest()
        hd = HistoryDisplay()
        return [st, rlr, es]

Hiperparametros

In [0]:
compile_params = {
    'maxlen': maxlen,
    'vocab': len(vocab),
    'emb_feats': 100,
    'rnn_hidden': 256
}

In [34]:
lm = LM()

lm_model = lm.compile_model(params=compile_params)






Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5, 100)            7900      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stack (CuDNNLSTM)            (None, 256)               788480    
_________________________________________________________________
dense_1 (Dense)              (None, 79)                20303     
Total params: 1,549,867
Trainable params: 1,549,867
Non-trainable params: 0
_________________________________________________________________


Entrenar

In [0]:
train_params = {
    'epochs': 150,
    'batch_size': 512,
    'shuffle': True,
    'vocab': len(vocab),
    'maxlen': maxlen,
}

In [0]:
SAMPLE_EVERY = 4

In [37]:
#Utilizando GPU si no el algoritmo no funciona
lm.train(lm_model, data=data_train, params=train_params)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/150





349386/349386 [==============================] - 27s 77us/step - loss: 2.5238 - acc: 0.2815
Epoch 2/150
349386/349386 [==============================] - 11s 32us/step - loss: 1.9406 - acc: 0.4437
Epoch 3/150
349386/349386 [==============================] - 11s 32us/step - loss: 1.6596 - acc: 0.5250
Epoch 4/150
349386/349386 [==============================] - 11s 33us/step - loss: 1.5165 - acc: 0.5599
Epoch 5/150
349386/349386 [==============================] - 11s 32us/step - loss: 1.4295 - acc: 0.5813
----- diversity: 0.2
<SOS>thank better state secretary stand andrew said protect make america great again! https://t.co/nggggut tax cut reported make america great again! https://t.co/wjnej
----- diversity: 0.6
<SOS>thank impact longer. make america, working congratulation honor politically support get first get anothers stop today offer condolences terrible flagation last no